<a href="https://colab.research.google.com/github/Suraj209211/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/blob/main/RF_DL_Hybrid_Mineral_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
data_new = pd.read_csv('/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/NEW_DATA/CSV_OUTPUTS/finaldata_geophysicalchemical.csv')

In [ ]:
data_new

In [ ]:
X_new = data_new.drop(columns=['X', 'Y', 'PROBABILITY'])
y_new = data_new['PROBABILITY']

In [ ]:
scaler = StandardScaler()
X_scaled_new = scaler.fit_transform(X_new)


In [ ]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_scaled_new, y_new, test_size=0.2, random_state=42)


In [ ]:
rf_advanced_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_advanced_model.fit(X_train_new, y_train_new)


In [ ]:
important_features = rf_advanced_model.feature_importances_

In [ ]:
top_features_idx = important_features.argsort()[-10:][::-1]
X_train_selected = X_train_new[:, top_features_idx]
X_test_selected = X_test_new[:, top_features_idx]

In [ ]:
dl_model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_selected.shape[1],)),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])

In [ ]:
dl_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

history = dl_model.fit(X_train_selected, y_train_new, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Import necessary libraries for plotting
import matplotlib.pyplot as plt

# Plot the training accuracy and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot the training loss and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
dl_loss, dl_accuracy = dl_model.evaluate(X_test_selected, y_test_new)

In [ ]:
print(f"Deep Learning Model Accuracy: {dl_accuracy}")

In [ ]:
# Save the deep learning model
dl_model.save('/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/MODEL/rf_dl_hybrid_mineral_dl_model.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
loaded_model = load_model('/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/MODEL/rf_dl_hybrid_mineral_dl_model.h5')

In [ ]:
# Evaluate the model on the test data
test_loss, test_accuracy = dl_model.evaluate(X_test_selected, y_test_new)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# Make predictions on the test data
predictions = dl_model.predict(X_test_selected)

# Convert probabilities to binary class labels (assuming binary classification)
predicted_labels = (predictions > 0.5).astype("int32")

# Display some of the predicted labels along with actual labels
for i in range(10):  # Display first 10 predictions
    print(f"Predicted: {predicted_labels[i][0]}, Actual: {y_test_new.iloc[i]}")

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Make predictions on the test data
predictions = dl_model.predict(X_test_selected)

# Convert probabilities to binary class labels
predicted_labels = (predictions > 0.5).astype("int32")

# Create confusion matrix
conf_matrix = confusion_matrix(y_test_new, predicted_labels)

# Plot confusion matrix using heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

# Load the pretrained model from the file
model_path = '/content/drive/MyDrive/PREDICTIVE MODULE OF MINERAL EXPLORATION/PMOMEX---PREDICTIVE-MODEL-OF-MINERAL-EXPLORATION/CODE/MODEL/rf_dl_hybrid_mineral_dl_model.h5'
loaded_model = load_model(model_path)

# Display the model summary to understand its architecture
loaded_model.summary()


In [ ]:
import numpy as np

# Assuming you have your test data: X_test and y_test
# Evaluate the model
test_loss, test_accuracy = loaded_model.evaluate(X_test, y_test, verbose=2)

# Print the accuracy
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

# Save the accuracy to a text file
with open('model_accuracy.txt', 'w') as f:
    f.write(f'Test Accuracy: {test_accuracy * 100:.2f}%\n')
